In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5

from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat: 
    df[feat] = factorized_values
  else: 
    df[feat + SUFFIX_CAT] = factorized_values

In [7]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

##Decision Tree

In [12]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.81093847179, 75.95212671816309)

##Random Forest

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18864.27843495869, 38.24233578175403)

##XGBoost

In [18]:
xgb_params = {
  'max_depth' : 5, 
  'n_estimators' : 50,
  'learning_rate' : 0.1, 
  'seed' : 1  
}
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[21:18:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:18:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:19:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [19]:
m = xgb.XGBRegressor(**xgb_params)

m.fit(X, y)
imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[21:23:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [32]:
feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji__cat','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
len(feats)

20

In [22]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[22:09:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:09:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:09:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [40]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
new_feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), new_feats)

[22:46:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:46:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:46:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11364.016578827623, 80.05911838700821)

In [49]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )
feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[22:54:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:54:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:54:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9724.786775066808, 84.17027602984261)

In [52]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')) )
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )

feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[23:02:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:02:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:02:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9570.7287270162, 87.36868463251476)

In [0]:
!git config --global user.email "maciej.michalczak@gmail.com"
!git config --global user.name "Maciej"

In [54]:
!ls -la

total 109
drwx------ 2 root root  4096 Mar  2 22:58 data
-rw------- 1 root root 42668 Mar  3 23:10 day2_visualisation.ipynb
-rw------- 1 root root 24052 Mar  4 22:28 day3_simple_model.ipynb
-rw------- 1 root root 31246 Mar  5 23:03 day4.ipynb
drwx------ 4 root root  4096 Mar  2 22:57 .git
-rw------- 1 root root  1799 Mar  2 22:57 .gitignore
-rw------- 1 root root  1064 Mar  2 22:57 LICENSE
-rw------- 1 root root    88 Mar  2 22:57 README.md
